# Implementação

In [1]:
import numpy as np
import pandas as pd
import random
from time import process_time
import statistics

## Codificação para a solução

In [2]:
# solução é um array de tamanho 8, onde cada posição representa uma coluna e seu valor representa a linha

def generate_random_solution():
    return [random.randint(0, 7) for i in range(0, 8)]

In [3]:
# codificação decimal para codificação binária

def to_binary(decimal_solution):
    return [bin(position)[2:].zfill(3) for position in decimal_solution]

In [4]:
# codificação binária para codificação decimal

def to_decimal(binary_solution):
    return [int(position, 2) for position in binary_solution]

In [5]:
# gerar população de k indivíduos já em binário

def generate_population(k):
    return [to_binary(generate_random_solution()) for i in range(0, k)]

## Função Objetivo

In [6]:
# quantidade de pares de rainhas se atacando

def calculate_fitness(solution):
    collisions = 0
    
    # iterando sobre cada rainha
    for column in range(0, 8):
        line = solution[column]

        # contar colisões com a rainha atual na mesma linha
        collisions += solution.count(line) - 1
        
        # contar colisões com a rainha atual na diagonal superior esquerda
        for i, j in zip(range(line-1, -1, -1), range(column-1, -1, -1)):
            if solution[j] == i:
                collisions += 1

        # contar colisões com a rainha atual na diagonal inferior esquerda
        for i, j in zip(range(line+1, 8, +1), range(column-1, -1, -1)):
            if solution[j] == i:
                collisions += 1
        
        # contar colisões com a rainha atual na diagonal superior direita
        for i, j in zip(range(line-1, -1, -1), range(column+1, 8, +1)):
            if solution[j] == i:
                collisions += 1
        
        # contar colisões com a rainha atual na diagonal inferior direita
        for i, j in zip(range(line+1, 8, +1), range(column+1, 8, +1)):
            if solution[j] == i:
                collisions += 1

    return collisions//2

## Operadores

In [7]:
# estratégia da roleta, quanto maior o fitness maior a chance de ser escolhido como pai

def select_parents(population, fitnesses):
    sum_fitnesses = sum(fitnesses)
    probability_distribution = [fit/sum_fitnesses for fit in fitnesses] # distr. de prob. para as fitness

    # são escolhidos 2 indexes dentre os k indivíduos, sem repetição, de acordo com a distribuição de probabilidades
    parents_index = np.random.choice(len(population), 2, p=probability_distribution, replace=False)
    
    return [population[i] for i in parents_index]

In [8]:
# estratégia do ponto de corte, onde o crossover tem uma probabilidade cross_rate de acontecer

def crossover(parents, cross_rate):

    if random.uniform(0, 1) <= cross_rate: # se o cruzamento for possível, trocar material genético por ponto de corte
        children = []
        
        cutoff = random.randint(1, 7)
        
        child_1 = parents[0][0:cutoff] + parents[1][cutoff:]
        child_2 = parents[1][0:cutoff] + parents[0][cutoff:]
        
        children.append(child_1)
        children.append(child_2)
        
        return children
        
    return parents # se o cruzamento não for possível, os pais se tornam filhos

In [9]:
# estratégia do bit flip, onde um dos 24 bits de um filho é trocado

def mutate(child, mutate_rate):
    
    if random.uniform(0, 1) <= mutate_rate: # se a mutação for possível, alterar material genético por bit flip
        mutated_child = []
        
        unified_genes = list("".join(child)) # unificar todo o material genético em uma lista de 24 bits
        bit = random.randint(0, 23) # bit a ser alterado
        
        # troca do bit
        if unified_genes[bit] == "0":
            unified_genes[bit] = "1"
        else:
            unified_genes[bit] = "0"
        
        mutated_string = "".join(unified_genes) # junta os bits em uma única string
        
        for i in range(0, 24, 3):
            mutated_child.append(mutated_string[i:i+3]) # divide os bits de 3 em 3, obtendo a forma original
        
        return mutated_child

    return child # se a mutação não for possível, o filho não se altera

In [10]:
# estratégia elitista, onde os k indivíduos com as menores fitness sobrevivem

def select_survivors(map_fitness_to_individual, k):
    sorted_map = sorted(map_fitness_to_individual) # ordenando as fitness em ordem crescente
    return sorted_map[:k] # retornando os k sobreviventes

## Algoritmo Genético

In [11]:
# recebe o tamanho da população k, a taxa de crescimento cross_rate, a taxa de mutação mutate_rate e máximo de gerações gen

def genetic_algorithm(k, cross_rate, mutate_rate, gen):
    population = generate_population(k)
    fitnesses = [calculate_fitness(to_decimal(individual)) for individual in population] # fitness para cada indivíduo
    best_solution = min(zip(fitnesses, population)) # 2-tupla que armazena a menor fitness e o melhor indivíduo da população
    
    while best_solution[0] > 0 and gen > 0:
        # seleção dos pais
        parents = select_parents(population, fitnesses)
        
        # cruzamento
        children = crossover(parents, cross_rate)

        # mutação e avaliação dos filhos
        mutated_children = [mutate(child, mutate_rate) for child in children]
        fitnesses_children = [calculate_fitness(to_decimal(individual)) for individual in mutated_children]
        
        # adicionando os filhos na população e suas fitness na lista de fitnesses
        population.extend(mutated_children)
        fitnesses.extend(fitnesses_children)
        
        # mapeando fitness aos respectivos individuos, k + 2 tuplas
        map_fitness_to_individual = zip(fitnesses, population)

        # selecionando as k melhores tuplas
        survivors = select_survivors(map_fitness_to_individual, k)
        
        # obtendo a melhor solução até o momento
        best_solution = min(survivors)
        
        # desfazendo o mapeamento
        fitnesses, population = [list(tup) for tup in zip(*survivors)]
        
        gen -= 1
    
    # retorna a melhor solução, sua fitness e a geração de parada
    return [best_solution[1], best_solution[0], 1000-gen]

# Execução

> Execute 50 vezes o algoritmo e apresente, em forma de tabela, a melhor solução encontrada em cada execução, o valor da função objetivo desta solução encontrada, o tempo de execução e o número da geração em que o algoritmo parou

In [12]:
execution = []
solutions = [] # soluções para cada uma das 50 execuções
fitnesses = [] # fitness para cada uma das 50 execuções
generations = [] # geração de parada para cada uma das 50 execuções
runtimes = [] # tempos de execução para cada uma das 50 execuções


# 50 execuções
for i in range(1, 51):
    execution.append(i) # número da execução
    start = process_time() # tempo da CPU antes da execução
    result = genetic_algorithm(20, 0.8, 0.03, 1000) # solução, fitness e geração de parada
    end = process_time() # tempo da CPU após a execução
    
    runtimes.append((end - start)*1000)
    solutions.append(result[0])
    fitnesses.append(result[1])
    generations.append(result[2])
    
df = pd.DataFrame({ "Runtime": execution,
                    "Best Solution": solutions,
                    "Fitness": fitnesses,
                    "Runtime (ms)": runtimes,
                    "Generation": generations})
df

,Runtime,Best Solution,Fitness,Runtime (ms),Generation
0,1,"[000, 101, 101, 010, 110, 011, 111, 100]",1,297.834493,1000
1,2,"[100, 001, 101, 000, 010, 110, 000, 011]",2,212.572509,1000
2,3,"[101, 010, 001, 110, 000, 011, 000, 100]",2,211.935033,1000
3,4,"[010, 101, 001, 100, 000, 011, 110, 010]",1,208.666921,1000
4,5,"[001, 011, 111, 010, 100, 010, 000, 101]",1,247.128773,1000
5,6,"[000, 011, 000, 111, 101, 001, 001, 100]",2,212.175686,1000
6,7,"[100, 000, 011, 101, 111, 001, 110, 010]",0,57.700558,272
7,8,"[000, 010, 100, 001, 111, 000, 011, 001]",2,222.602386,1000
8,9,"[001, 101, 000, 010, 100, 111, 011, 110]",1,223.381591,1000
9,10,"[010, 100, 010, 111, 101, 011, 000, 110]",1,201.594647,1000


> Calcular a média e o desvio padrão do valor da função objetivo do melhor indivíduo, do tempo de execução e o número da geração em que o algoritmo parou (três últimas colunas da tabela)

In [13]:
fitnesses_mean = statistics.mean(fitnesses)
fitnesses_stdev = statistics.stdev(fitnesses)
print("Fitnesses")
print("Mean: ", fitnesses_mean)
print("StDev: ± ", fitnesses_stdev)

runtimes_mean = statistics.mean(runtimes)
runtimes_stdev = statistics.stdev(runtimes)
print("\nRuntimes")
print("Mean: ", runtimes_mean, )
print("StDev: ± ", runtimes_stdev)

generations_mean = statistics.mean(generations)
generations_stdev = statistics.stdev(generations)
print("\nGenerations")
print("Mean: ", generations_mean, )
print("StDev: ± ", generations_stdev)

Fitnesses
Mean:  1.44
StDev: ±  0.5771145679064045

Runtimes
Mean:  211.01480349999994
StDev: ±  49.443398489605265

Generations
Mean:  966.42
StDev: ±  167.69774681167988


> Mostre, pelo menos, duas soluções distintas encontradas pelo algoritmo

In [14]:
# 5 melhores soluções encontradas

# mapeando fitness às respectivas soluções
map_fitness_to_solutions = zip(fitnesses, solutions)

# ordenando as fitness em ordem crescente
sorted_map = sorted(map_fitness_to_solutions)

# obtendo as 5 melhores soluções
top_five_solutions = sorted_map[:5]

for solution in top_five_solutions:
    print('-'*34)
    print("Solution: ", solution[1])
    print("Fitness: ", solution[0])

----------------------------------
Solution:  ['010', '101', '001', '100', '111', '000', '110', '011']
Fitness:  0
----------------------------------
Solution:  ['100', '000', '011', '101', '111', '001', '110', '010']
Fitness:  0
----------------------------------
Solution:  ['000', '010', '101', '111', '001', '011', '000', '110']
Fitness:  1
----------------------------------
Solution:  ['000', '011', '101', '010', '110', '001', '111', '100']
Fitness:  1
----------------------------------
Solution:  ['000', '011', '101', '111', '001', '110', '000', '010']
Fitness:  1
